In [12]:


import os
import csv
import datetime
import re
import sys
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import configparser

config = configparser.ConfigParser()

        
# Read the configuration file
config.read("config.ini")

skills = config.get("other", "skills", fallback="Default Value")
experience = config.get("other", "experience", fallback="3")
job_titles = config.get("other", "job_titles").split(",")
location = config.get("other", "location")

## options are 1, 3, 7, 15, 30 days | will take the max out of it automatically
job_age = max(config.get("other", "jobAge", fallback="").split(','))

## Applications that is not required to be applied for the respective Job Title !
exclude_keywords = config.get("other", "exclude_keywords", fallback="").split(",") 

my_info = dict(config.items("my_info"))
question_mappings = {
    "experience": experience,
    "first name": my_info.get        ("First", my_info.get("first", "")),
    "last name": my_info.get         ("Last", my_info.get("last", "")),
    "number": my_info.get            ("Number", my_info.get("number", "")),
    "phone": my_info.get             ("Number", my_info.get("number", "")),
    "contact": my_info.get           ("Number", my_info.get("number", "")),
    "website": my_info.get           ("Github", my_info.get("github", "")),
    "github": my_info.get            ("Github", my_info.get("github", "")),
    "current city": my_info.get      ("City", my_info.get("city", "")),
    "current location": my_info.get  ("City", my_info.get("city", "")),
    "expected annual": my_info.get   ("ECTC", my_info.get("ectc", "")),
    "expected salary": my_info.get   ("ECTC", my_info.get("ectc", "")),
    "expected ctc": my_info.get      ("ECTC", my_info.get("ectc", "")),
    "salary expectation": my_info.get("ECTC", my_info.get("ectc", "")),
    "join": my_info.get              ("notice_period", my_info.get("Notice Period", "")),
    "languages": my_info.get         ("languages", my_info.get("Languages", "")),
    "languages": my_info.get         ("languages", my_info.get("Languages", "")),
    "job change": my_info.get        ("job_change_reason", my_info.get("Job Change Reason", "")),
    "current ctc": my_info.get       ("CTC", my_info.get("ctc", "")),
    "current annual ctc": my_info.get("CTC", my_info.get("ctc", "")),
    "current ctc": my_info.get       ("CTC", my_info.get("ctc", "")),
    "current annual ctc": my_info.get("CTC", my_info.get("ctc", "")),
    "expected annual": my_info.get   ("ECTC", my_info.get("ectc", "")),
    "expected salary": my_info.get   ("ECTC", my_info.get("ectc", "")),
    "expected ctc": my_info.get      ("ECTC", my_info.get("ectc", "")),
    "salary expectation": my_info.get("ECTC", my_info.get("ectc", "")),
    "notice period": my_info.get     ("notice_period", my_info.get("Notice Period", "")),
    "join": my_info.get              ("notice_period", my_info.get("Notice Period", "")),
    "when can you start": my_info.get("notice_period", my_info.get("Notice Period", "")),
    "when can you start": my_info.get("notice_period", my_info.get("Notice Period", "")),
    "university": my_info.get        ("university", my_info.get("University", "")),
    "school": my_info.get            ("university", my_info.get("School", "")),
    "institute": my_info.get         ("university", my_info.get("Institute", "")),
    "university": my_info.get        ("university", my_info.get("University", "")),
    "school": my_info.get            ("university", my_info.get("School", "")),
    "institute": my_info.get         ("university", my_info.get("Institute", "")),
    "skills": skills
}


applied_jobs = set()  

wait = WebDriverWait(driver, 10)

ctc_expectations = {
    1: "0to3",
    2: "3to6",
    3: "6to10",
    4: "10to15",
    5: "15to25",
    6: "25to50",
    7: "50to75",
}

location_to_city_type_gid = {
    1: {"mumbai": 134},
    2: {"chennai": 183},
    3: {"usa": 9044},
    4: {"Hyderabad": 17},
    5: {"bangalore": 97},
    6: {"delhi/NCR": 9508},
    7: {"Pune": 139},
    8: {"Kolkata": 232},
    9: {"Mumbai(All Areas)": 9509},
}



# Redirect SSL handshake error messages to nul (Windows)
if os.name == 'nt':
    sys.stderr = open(os.devnull, 'w')
# Redirect SSL handshake error messages to /dev/null (Linux/Mac)
if os.name == 'posix':
    sys.stderr = open('/dev/null', 'w')

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")

In [8]:
# Create a new instance of the Chrome web driver
driver = webdriver.Chrome(options=chrome_options)

def accept_cookies():
    actions = ActionChains(driver)
    actions.move_by_offset(1, 1).click().perform()
    
def refresh_page():
    driver.refresh()
    accept_cookies()  # Accept cookies after refreshing

def login_portal():
    
    # Navigate to the Naukri website
    driver.get("https://www.naukri.com/nlogin/login")
    accept_cookies()
    

    username = "tusharmalhan@gmail.com"
    password = "tusharmalhan@21"


    sleep(2)
    # Enter your login credentials
    driver.find_element(By.ID, "usernameField").send_keys(username)
    driver.find_element(By.ID, "passwordField").send_keys(password)
    driver.find_element(By.ID, "passwordField").send_keys(Keys.ENTER)

    sleep(4)

login_portal()


if re.search(r'Invalid', driver.page_source, re.IGNORECASE):
    driver.refresh()
    login_portal()
else:
    print('\nLogged in Successfully to Naukri Portal \n')


Logged in Successfully to Naukri Portal 



In [15]:

def get_element_xpath(element):
    return str(driver.execute_script("function absoluteXPath(element) {var comp, comps = [];var parent = null;var xpath = '';var getPos = function(element) {var position = 1, curNode;if (element.nodeType == Node.ATTRIBUTE_NODE) {return null;}for (curNode = element.previousSibling; curNode; curNode = curNode.previousSibling) {if (curNode.nodeName == element.nodeName) {++position;}}return position;};if (element instanceof Document) {return '/';}for (; element && !(element instanceof Document); element = element.nodeType == Node.ATTRIBUTE_NODE ? element.ownerElement : element.parentNode) {comp = comps[comps.length] = {};switch (element.nodeType) {case Node.TEXT_NODE:comp.name = 'text()';break;case Node.ATTRIBUTE_NODE:comp.name = '@' + element.nodeName;break;case Node.PROCESSING_INSTRUCTION_NODE:comp.name = 'processing-instruction()';break;case Node.COMMENT_NODE:comp.name = 'comment()';break;case Node.ELEMENT_NODE:comp.name = element.nodeName;break;}}for (var i = comps.length - 1; i >= 0; i--) {comp = comps[i];xpath += '/' + comp.name.toLowerCase();if (comp.hasOwnProperty('position')) {xpath += '[' + comp.position + ']';}}return xpath;}return absoluteXPath(arguments[0]);", element))


In [16]:

def naukri_portal(experience, job_title, location, job_age, exclude_keywords):
    
    for positions in job_title:

        # import pdb;pdb.set_trace()
        job_title = '-'.join(positions.split(' ')).lower()
        job_url = f"https://www.naukri.com/{job_title}-jobs?experience={experience}"

        if location is None or location =="None":
            location_preference = [
                location_to_city_type_gid[4].get("Hyderabad"), 
                location_to_city_type_gid[5].get("bangalore"),
                location_to_city_type_gid[5].get("Mumbai(All Areas)")
                ]
            job_url = f"https://www.naukri.com/{job_title}-jobs?experience={experience}"
            for location in location_preference:
                if location:
                    job_url += f"&cityTypeGid={location}"
        else:
            job_url +="&wfhType=2"

        # Add the CTC expectations to the job URL
        ctc_expectations_values = [ctc_expectations[5], ctc_expectations[6], ctc_expectations[4]]
        for ctc_expectations_value in ctc_expectations_values:
            job_url += f"&ctcFilter={ctc_expectations_value}"

        job_url += f"&jobAge={job_age}"

    
        sleep(2)
        driver.get(job_url)
        accept_cookies()

        # Get the current window handle.
        current_window_handle = driver.current_window_handle
        sleep(1)
        
        
        try:
            job_links = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "list")))
            try:
                sleep(1)
                all_jobs = wait.until(EC.visibility_of_all_elements_located((By.TAG_NAME, "article")))
            except:
                all_jobs = [link.find_elements(By.TAG_NAME, "article") for link in job_links][0]
            print(f"\nLets initiate Automatic Job Search on Naukri Portal for {positions.upper()} position \n")
        
        except Exception as e :
            print(f"\n\t Apologies, No Jobs found for Position {positions.upper()} as per your requirement \n ")
            continue
    
        # Create a DataFrame to store the job data
        job_data = []

        for job in all_jobs:

            if job in applied_jobs:
                continue
    
            import pdb;pdb.set_trace()
            sleep(1)
        
            try:
                job.click()
                
            except:
                sleep(1)
                print("\nStaleElementReferenceException occurred in job click. Retrying...\n")
                import pdb;pdb.set_trace()
                sleep(2)
                try:
                    driver.refresh()
                    driver.get(job_url)
                    accept_cookies()
                    sleep(3.5)
                    job.click()
                except:
                    driver.quit()
                    print("\nCant proceed with the stale element\n")
                    df.to_csv(csv_file_path, index=False)

                    # Print a success message
                    print(f"\nJob data has been saved to {csv_file_path}\n")
                    
                    # job_data.append([job_title, '', f"Not Applying because of keyword mentioned {keyword.lower()}", '', '', "No URL", '', '', ''])
                    # driver.refresh()
                    # driver.get(job_url)
                    # accept_cookies()
                    # sleep(3.5)
                    # job.click()
                
            accept_cookies()
            sleep(2)
            ## Switch to the new tab.
            for window_handle in driver.window_handles:
                if window_handle != current_window_handle:
                    driver.switch_to.window(window_handle)
        
            ## Switch to the new tab one liner
            # [ driver.switch_to.window(window_handle) for window_handle in driver.window_handles if window_handle != current_window_handle ]

                

            try:
                try:
                    html_content = driver.find_element(By.CLASS_NAME, "job-desc").text
                    recruiters_details = re.findall(r'\+91\d{10}', html_content)
                    if not recruiters_details:
                        recruiters_details = driver.find_element(By.CLASS_NAME, "rec-details").text
                    else:
                        import pdb;pdb.set_trace()
                except: 
                    # print('Look for phone number in description ')
                    recruiters_details = ''.join(recruiters_details)
            except NoSuchElementException:
                recruiters_details = "No Recruiters Details Found"

                
                
                
            try:
                try:job_title = driver.find_element(By.CLASS_NAME, "jd-header-title").text
                except: job_title = driver.title
                apply_flag = True
                for keyword in exclude_keywords:
                    if keyword.lower() in [i.lower().replace('(', '').replace(')', '') for i in job_title.split(' ')]:
                        print('Not applying for:', job_title, f' as keyword "{keyword.lower()}" is mentioned in the job title\n')
                        job_data.append([job_title, '', f"Not Applying because of keyword mentioned {keyword.lower()}", '', '', "No URL", '', '', ''])
                        apply_flag = False
                        break

                if not apply_flag:
                    sleep(1)
                    driver.close()
                    driver.switch_to.window(current_window_handle)
                    sleep(1.5)
                    continue

                company_name_raw = driver.find_element(By.CLASS_NAME, "jd-header-comp-name")
                company_name = company_name_raw.find_element(By.TAG_NAME, "a").text
                salary = driver.find_element(By.CLASS_NAME, "salary").text
                print(f'\nClicking the Next Job Title {job_title}  Button...\t\n')
                job_data.append([job_title, company_name, "Applied Successfully", salary, recruiters_details ,"No URL", '','',''])
            
            except Exception as e :

                sleep(2)
                print(f"\nFound some issues in getting meta info from the job we need to click \n" )

                print("\n############## PORTAL TO APPLY DIRECTLY ON THE WEBSITE ############# \n")
                
                # import pdb;pdb.set_trace()

                company_name = driver.title.split('-')[1].split(',')[0]
                result = manual_apply(job_data, job_title, company_name)
                if result == "Career Section Unavailable":
                    job_data.append([job_title, company_name, "Career Section Unavailable", "Not Disclosed by Recruiter", [], driver.current_url, '', '', ''])
                elif result == "Issue in Web Page":
                    job_data.append([job_title, company_name, "Issue in Web Page", "Not Disclosed by Recruiter", [], driver.current_url, '', '', ''])
                else:
                    job_data.append(["job_title", "company_name", "Apply manually on company's portal", "  Not Disclosed by Recruiter ", recruiters_details, driver.current_url, '','',''])
                driver.close()
                driver.switch_to.window(current_window_handle)
                sleep(3) 
                continue
            
            ### Apply for the job        
            try:
                try:
                    link_text = driver.find_element(By.CLASS_NAME, "apply-button")
                except NoSuchElementException:
                    link_text = driver.find_element(By.CLASS_NAME, "already-applied")
                    print(f"\nAlready applied to {company_name} for the title {job_title}.\n ")
                    job_data.append([job_title, company_name, "Already applied", salary, recruiters_details, "No URL", '','',''])
                    driver.close()
                    driver.switch_to.window(current_window_handle)
                    continue

            except NoSuchElementException:
                print('\n############## PORTAL TO APPLY DIRECTLY ON THE WEBSITE ############# \n')
                
                try:
                    link_text = driver.find_element(By.CLASS_NAME, "company-site-button")
                    link_text.click()
                except:
                    # link_text = driver.find_element(By.CLASS_NAME, "walkin-button")
                    print('\nWalkin position\n')
                sleep(1.5)
                company_website = driver.current_url

                driver.close()
                # Switch to company website
                [ driver.switch_to.window(window_handle) for window_handle in driver.window_handles if window_handle != current_window_handle ]

                
            

                import pdb; pdb.set_trace()
                result = manual_apply(job_data, job_title, company_name)
                if result == "Career Section Unavailable":
                    job_data.append([job_title, company_name, "Career Section Unavailable", "Not Disclosed by Recruiter", [], driver.current_url, '', '', ''])
                elif result == "Issue in Web Page":
                    job_data.append([job_title, company_name, "Issue in Web Page", "Not Disclosed by Recruiter", [], driver.current_url, '', '', ''])
                else:
                    job_data.append(["job_title", "company_name", "Apply manually on company's portal", "  Not Disclosed by Recruiter ", recruiters_details, driver.current_url, '','',''])
                driver.close()
                driver.switch_to.window(current_window_handle)
                sleep(3)
                continue
            
            sleep(1)
            
            print('Clicking Apply button... \t\n')
            
            
            try:    link_text.click()
            except: sleep(2.5);link_text.click()

            import pdb;pdb.set_trace()
            print(' clicking on Update and apply button \n\t')

            sleep(2)
            
            if 'This company does not require you to apply again as you have previously' in driver.page_source:
                print(f"\nJob submission failed for the title {job_title} to {company_name}, as company has blocked you\n")
                job_data.append([job_title, company_name, "Company Banned You, LOL ! ", salary, recruiters_details ,driver.current_url, '','',''])
                ### Switch back to original tab, as job cant be posted
                driver.close()
                driver.switch_to.window(current_window_handle)
                continue

            filling_questionnaire(company_name, job_data)

        
            ### After filling the Quesstionnaire form , if still you find questionnaire, issue in Page
            qus_form = None
            try:
                sleep(3)
                qus_form = driver.find_element(By.NAME, "qusForm")
            except:
                try:
                    qus_form = driver.find_element(By.NAME, "qupForm")
                except: ...
            if qus_form:
                print(f"\nApply Manually \nJob submission failed for the title {job_title} to {company_name}.\n")
                job_data.append([job_title, company_name, "Submission Failed ", salary, recruiters_details ,driver.current_url, '','',''])
                # import pdb;pdb.set_trace()
                driver.close()
                driver.switch_to.window(current_window_handle)
                continue
            
            sleep(2)
            try:success_element = driver.find_element(By.CLASS_NAME, "apply-status-header.green")
            except:...
            daily_quota_limit = None
            
            try:daily_quota_limit = driver.find_element(By.XPATH, "//span[contains(text(),'Your daily quota has been expired.')]")
            except NoSuchElementException: ...
            if daily_quota_limit:
                driver.close()
                print("\nThe limit to apply to all your specific jobs has been reached!")
                print('Catch yeah tommorrow')
                break
            sleep(1.2)
            if success_element:
                print(f"\nJob submitted successfully for the title {job_title} to {company_name}\n\n")
                sleep(1)
            else:
                try:
                    if link_text.text !='Applied':
                        # import pdb;pdb.set_trace()
                        print(f"Job submission failed for the title {job_title} to {company_name}.\n")
                        job_data.append([job_title, company_name, "Submission Failed ", salary, recruiters_details ,driver.current_url, '','',''])
                    else: ...
                except:
                    driver.refresh()
                    driver.back()

            # close the driver for current job portal
            driver.close()

            # add the job to the set
            applied_jobs.add(job)

            # Switch back to the original tab.
            driver.switch_to.window(current_window_handle)

        # Generate a timestamp as the sheet name
        timestamp = datetime.now().strftime("%d_%m__%H__%M%S")
    
        # Create the main directory
        main_directory = "Naukri_Jobs_cv"
        if not os.path.exists(main_directory):
            os.makedirs(main_directory)

        # Create the directory for the current date if it doesn't exist
        directory_name = datetime.now().strftime("%Y-%m-%d")
        directory_path = os.path.join(main_directory, directory_name)
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)

        # Construct the full file path with the directory and file name
        csv_file_path = os.path.join(directory_path, f"job_data_{timestamp}.csv")

        # Create a DataFrame from the job data
        df = pd.DataFrame(job_data, columns=["Job Title", "Company Name", "Status", "CTC", "Recruiters Details" ,"URL", "question_text", "input type", "user input text"])
        
    
        df.to_csv(csv_file_path, index=False)

        # Print a success message
        print(f"\nJob data has been saved to {csv_file_path}\n")

    driver.quit()

naukri_portal(experience, job_titles, location, job_age, exclude_keywords)







Lets initiate Automatic Job Search on Naukri Portal for PYTHON position 

> c:\users\tusharmalhan\appdata\local\temp\ipykernel_6132\124368119.py(61)naukri_portal()

*** NameError: name 'o' is not defined
*** Newest frame
> c:\users\tusharmalhan\appdata\local\temp\ipykernel_6132\124368119.py(63)naukri_portal()

*** Newest frame
> c:\users\tusharmalhan\appdata\local\temp\ipykernel_6132\124368119.py(64)naukri_portal()

experience = '3'
job_title = 'python'
location = None
job_age = '7'
exclude_keywords = ['adobe', '3d', '2d', 'android', 'blockchain', 'dot', 'salesforce', 'frontend', 'angular', 'c++', 'game', ' azure', 'c#', 'java', 'UI', 'UX', 'react', 'reactjs', 'vue', 'ember', 'php', 'Applications', 'sales', 'asp', 'asp.net', '.net', 'flutter', 'C#.Net']
*** Newest frame
*** NameError: name 'o' is not defined
*** NameError: name 'o' is not defined
--Return--
None
> c:\users\tusharmalhan\appdata\local\temp\ipykernel_6132\124368119.py(115)naukri_portal()

UnboundLocalError: cannot access

In [16]:
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np


data = [
    'White / Caucasian',
    'Hispanic, Latino, or Spanish origin',
    'Black or African American',
    'Asian',
    'Native Hawaiian or other Pacific Islander',
    'Indigenous Peoples, First Nations, Native American, or Alaska Native',
    'Middle Eastern or North African',
    'Some other race, ethnicity, or origin'
]

# Creating labels for the data
labels = [f'ethnicity_{i}' for i in range(len(data))]

# Creating a list of tuples where each tuple is (text, label)
training_data = [(item, label) for item, label in zip(data, labels)]

# Creating a bag-of-words model
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([item[0] for item in training_data])
y = np.array([item[1] for item in training_data])

# Training a Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X, y)

# Save the model and vectorizer
joblib.dump(vectorizer, 'training_data/vectorizer_ethnicity.joblib')
joblib.dump(classifier, 'training_data/classifier_ethnicity.joblib')



['training_data/classifier_ethnicity.joblib']

In [21]:
# Function to load and use the model
def identify_ethnicity_in_list(input_list):
    loaded_vectorizer = joblib.load('training_data/vectorizer_ethnicity.joblib')
    loaded_classifier = joblib.load('training_data/classifier_ethnicity.joblib')
    
    for text in input_list:
        X_test = loaded_vectorizer.transform([text])
        prediction = loaded_classifier.predict(X_test)

        if prediction:
            print("What is your ethnicity?")
            break

# Example usage:
input_strings = ['White', 'Asian', 'Native', 'Indigenous']
identify_ethnicity_in_list(input_strings)


['ethnicity_0']
What is your ethnicity?


NameError: name 'X_test' is not defined

In [2]:
import openpyxl
from openpyxl.styles import Font

workbook_expenses_file = r"C:\Users\gyala\Documents\expenses.xlsx"

# Create a new workbook
workbook = openpyxl.Workbook()
sheet = workbook.active

# Define the data
data = {
    "Budget /monthly": 190000,
    "mom": 20000,
    "rent": 18000,
    "worker": 3000,
    "elec+bil": 1000,
    "petrol": 1000,
    "phone": 500,
    "youtube": 170,
    "Total Expenses": "43,670",
    "Remaning": 146330,
    "food": 'Gonna calculate'
}

# Set the font style for specific columns
bold_font = Font(bold=True, size=17)
bold_columns = ["Budget /monthly", "Total Expenses", "Remaning"]

# Populate the sheet with the data
for row, (key, value) in enumerate(data.items(), start=1):
    sheet.cell(row=row, column=1, value=key)
    sheet.cell(row=row, column=2, value=value)
    if key in bold_columns:
        sheet.cell(row=row, column=1).font = bold_font
        sheet.cell(row=row, column=2).font = bold_font

# Save the workbook
workbook.save(f"{workbook_expenses_file}")


import openpyxl

# Load the workbook
workbook = openpyxl.load_workbook(workbook_expenses_file)
sheet = workbook.active

# Loop through the rows and print the data
for row in sheet.iter_rows(values_only=True):
    for cell in row:
        print(cell, end="\t")
    print()

# Close the workbook
workbook.close()


Budget /monthly	190000	
mom	20000	
rent	18000	
worker	3000	
elec+bil	1000	
petrol	1000	
phone	500	
youtube	170	
Total Expenses	43,670	
Remaning	146330	
food	Gonna calculate	


In [29]:
import threading
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def search_google(keyword):
    driver = webdriver.Chrome()
    driver.get("https://www.google.com/")

    search_box = driver.find_element(by=By.NAME, value="q")
    search_box.send_keys(keyword)
    search_box.submit()
    time.sleep(5)
    driver.get('https://bard.google.com/chat/cb787cc14626e4fc')

keywords = ["wwe", "machine learning", "data science"]

threads = []
for keyword in keywords:
    thread = threading.Thread(target=search_google, args=(keyword,))
    threads.append(thread)

for thread in threads:
    thread.start()
    thread.name = 'child'

    print(f"Starting thread {thread.getName()}")


for thread in threads:
    thread.join()



C:\Users\gyala\AppData\Local\Temp\ipykernel_13692\59630985.py:29: DeprecationWarning: getName() is deprecated, get the name attribute instead
  print(f"Starting thread {thread.getName()}")


Starting thread child
Starting thread child
Starting thread child


PASSING USER INFO TO A PARTICULAR WEBSITE


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import re
import datetime
import time
from time import sleep
from selenium import webdriver

driver = webdriver.Chrome()  # You may need to provide the path to your Chrome WebDriver

def get_web():
    # url = "https://tresvistacareers.turbohire.co/surveys?token=SH3RVxWEbmWHSWgVU1ELI8fN8zzngUxfZ0Zmz5pVKVIDOsWVyUJn2d5XyNxRN8SnQuESBEcCgQA4Uw9_B67gMcnzs7ja1Qxmgtoclyp_4M%2Fj1zp55LRP_jhCNveetDQCW9zfGZHBippuDEVgDAZX1TYoLeNN1vp_20mZYPtwmhs=&company=nGouE9BaVHwVrgRUlwh3cqVB7jhQgMBxiNhOiYAT2%2Fc0JVdW1xICQmmiFoP4uAlZ&guest=tusharmalhan@gmail.com"
    url = 'https://share.hsforms.com/1zMVWcmmfR9aqwK4voZduOAnk8rr'
    # Initialize the Selenium WebDriver

    try:
        # Open the URL
        driver.get(url)
        time.sleep(2)  # Wait for the page to load



    finally:
        import pdb;pdb.set_trace()
        initialize_information()
        # Close the browser and quit the driver
        # driver.quit()


def initialize_information():


    # my_info = {"Given": "Tushar", "First": "Tushar", "Family": "Malhan", "Last": "Malhan", "Full Name": "Tushar Malhan", "Local given Name": "Tushar", "Address": "Khargar Laxmi Nivas", "Email": "tusharmalhan@gmail.com", "city": "Navi Mumbai", "Postal code": "400614", "Phone Number": "7814891872", "phone extension": "+91", "password": "Test@123", "Verify new": "Test@123", "LinkedIn Profile": "https://www.linkedin.com/in/tushar-malhan-9981841ab", "Website": "https://github.com/tushar2malhan", "Github": "https://github.com/tushar2malhan"}

    my_info = {
    "Given": "Tushar",
    "First": "Tushar",
    "Family": "Malhan",
    "Last": "Malhan",
    "Full Name":"Tushar Malhan",
    "Local given Name": "Tushar",
    "Address": "Khargar Laxmi Nivas",
    "Email": "tusharmalhan@gmail.com",
    "city": "Navi Mumbai",
    "Postal code": "400614",
    "Phone Number": "7814891872",
    "phone extension": "+91",
    "password":"Test@123",
    "Verify new":"Test@123",
    "LinkedIn Profile":"https://www.linkedin.com/in/tushar-malhan-9981841ab",
    "Website":"https://github.com/tushar2malhan",
    "Github":"https://github.com/tushar2malhan"
    }
    
    
    ### one liner
    # matched_label = next((label for key in my_info.keys() for label in driver.find_elements(By.XPATH, '//label|//*[text()]') if re.search(rf"({'|'.join(map(re.escape, key.split()))})", label.text, re.IGNORECASE)), None)
    ### one liner
    # !for key in my_info.keys(): key_words = key.split(); pattern = re.compile(rf"({'|'.join(map(re.escape, key_words))})", re.IGNORECASE); labels = driver.find_elements(By.XPATH, '//label'); matched_label = None; for label in labels: if pattern.search(label.text): matched_label = label; break; if matched_label: input_element = driver.find_element(By.XPATH, f"//label[text()='{matched_label.text}']/following-sibling::input"); value = input_element.get_attribute('value'); print(f"Key: {key}  Value: {value}"); else: label_text = input(f"No label found for key '{key}'. Please enter the corresponding label text: "); print(f"{key}: {label_text}")

    sleep(1.5)
    # # Iterate over each key in the my_info dictionary
    # for key, value in my_info.items():
        
    #     key_words = key.split()
    #     # Create a regular expression pattern to match the key or its associated value
    #     pattern = re.compile(rf"({'|'.join(map(re.escape, key_words))})", re.IGNORECASE)
        
    #     # Find the label element that matches the pattern
    #     labels = driver.find_elements(By.XPATH, '//label|//*[text()]') 
    #     matched_label = None
    #     for label in labels:
    #         if pattern.search(label.text):
    #             matched_label = label
    #             break
    #     if matched_label: # If a matching label is found, retrieve the associated input value
    #         import pdb;pdb.set_trace()
    #         try:
    #             label = driver.find_element(By.XPATH, f"//label[contains(text(), '{matched_label.text.split('(')[0]}')]")
    #             input_element = label.find_element(By.XPATH, "./following-sibling::div//input")
    #         except NoSuchElementException:
    #             try:
    #                 input_element = driver.find_element(By.XPATH, f"//input[@id='{matched_label.get_attribute('for')}']")
    #             except NoSuchElementException:
    #                 input_elements = driver.find_elements(By.TAG_NAME, 'input')
    #                 for element in input_elements:
    #                     if element.get_attribute('type') != 'hidden':
    #                         input_element = element
    #                         break
    #         input_element.clear()
    #         try:
    #             input_element.send_keys(value)
    #         except StaleElementReferenceException:
    #             label = driver.find_element(By.XPATH, f"//label[contains(text(), '{matched_label.text.split('(')[0] }')]")
    #             input_element = label.find_element(By.XPATH, "./following-sibling::div//input")
    #             input_element.send_keys(value)

    #         print(f"\n For Key: {key}   \t Value given: {value}\n")
    #     else:
    #         # Prompt the user to input the label text manually
    #         # label_text = input(f"No label found for key '{key}'. Please enter the corresponding label text: ")
    #         print(f"No label found for key {key}:   \n")

    import datetime

    def select_options():
        # Find input elements on the page
        input_elements = driver.find_elements(By.XPATH, '//input[@type!="hidden"]|//select[@style!="display:none"]')

        try:
            # try:
                # resume_button = driver.find_element(By.XPATH, "//button[contains(text(),'Attach')] | //a[contains(text(),'Attach')]")
            resume_button = driver.find_elements(By.XPATH, "//button[contains(translate(text(), 'abcdefghijklmnopqrstuvwxyz', 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'), 'ATTACH')] | //a[contains(translate(text(), 'abcdefghijklmnopqrstuvwxyz', 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'), 'ATTACH')]")
            # except:

            file_path = r"C:\Users\TusharMalhan\Documents\Tushar's_Resume.pdf"
            resume_button.send_keys(file_path)
            print(' Added file Attachment of Resume  ')
        except:
            print(' No file Attachment Found of Resume   ')



        # Iterate over the input elements
        import pdb;pdb.set_trace()
        for input_element in input_elements:
                # Try to find the label associated with the input element
            try:
                label = input_element.find_element(By.XPATH, './preceding::label[1]')
                label_text = label.text
            except:
                # If label not found, retrieve the text of the input element itself
                label_text = None

            # Determine the type of input element
            input_type = input_element.get_attribute('type')

            response = None
            
            if label_text:
                # Suppose input type starts with 'email', that's why 'or' case is placed here
                if (input_type == 'text') or input_type.lower() in [i.lower() for i in label_text.split()]:
                    print("TEXT OPTIONS")
                    import pdb;pdb.set_trace()

                    # Check if the label_text starts with or contains any key in my_info
                    for key in my_info:
                        if key and (key.lower() in (label_text.lower() if label_text else '')) :
                            # Check if the key already has a value assigned
                            if my_info.get(key) is not None:
                                response = my_info.get(key)
                            else:
                                if 'date' in input_type.lower():
                                    # Prompt the user to enter the date value
                                    date_input = input(f"Enter the date for '{label_text}' (YYYY-MM-DD): ")
                                    try:
                                        # Validate the date format
                                        datetime.datetime.strptime(date_input, "%Y-%m-%d")
                                        response = date_input
                                    except ValueError:
                                        print("Invalid date format. Please enter a date in YYYY-MM-DD format.")
                                else:
                                    # Prompt the user to enter the value
                                    response = input(f"{label_text}\t: ")

                            # Enter the response in the input element
                            child_input = input_element.find_element(By.CSS_SELECTOR, 'input')
                            child_input.send_keys(response)
                            break

                    else:
                        if 'date' in input_type.lower():
                            # Prompt the user to enter the date value
                            date_input = input(f"Enter the date for '{label_text}' (YYYY-MM-DD): ")
                            try:
                                # Validate the date format
                                datetime.datetime.strptime(date_input, "%Y-%m-%d")
                                response = date_input
                            except ValueError:
                                print("Invalid date format. Please enter a date in YYYY-MM-DD format.")
                        else:
                            # If no match found, prompt the user to type a response
                            response = input(f"Input for '{label_text}'\t: ")

                        # Enter the response in the input element
                        input_element.send_keys(response)
                        child_input = input_element.find_element(By.CSS_SELECTOR, 'input')
                        child_input.send_keys(response)


                elif input_type == 'select-one' :
                    
                    print("SELECTED OPTIONS")
                    
                    import pdb;pdb.set_trace()
                    # Get the available options for the select element
                    options = input_element.find_elements(By.XPATH, './option')
                    
                    # Display the label and available options to the user
                    if label_text is None:
                        parent_element = input_element.find_element(By.XPATH, '..')
                        parent_text = parent_element.text
                        print("Parent Text:", parent_text)
                    else:
                        print(label_text)
                    
                    for index, option in enumerate(options, start=1):
                        print(f"{index}. {option.text}")
                    
                    # Prompt the user to select an option
                    selected_option_index = int(input("Enter the option number: ")) - 1
                    
                    import pdb;pdb.set_trace()
                    # Select the chosen option
                    options[selected_option_index].click()
                           
            elif input_type == 'file':

                print("  FILE OPTIONS  ")

                import pdb;pdb.set_trace()
                # Prompt the user to enter the file path or provide it programmatically
                input_id = input_element.get_attribute('id')
                input_name = input_element.get_attribute('name')
                if input_id and 'resume' in input_id.lower() or input_name and 'resume' in input_name.lower():
                    attribute_name = 'resume'
                    file_path = r"C:\Users\TusharMalhan\Documents\Tushar's_Resume.pdf"
                else:
                    attribute_name = input_id or input_name
                    file_path = input(f"Enter the file path for {attribute_name}: \t {label_text} ")

                # Set the file path for the file input element
                input_element.send_keys(file_path)

                sleep(3)
                if 'Success' in driver.page_source or "Uploaded"  in driver.page_source :
                    print('\n Resume Uploaded Successfully \n')
            
            elif input_type == 'radio':
                
                print("RADIO OPTIONS")

                # Find the parent element
                parent_element = input_element.find_element(By.XPATH, '..')

                # Print the text of the parent element
                parent_text = parent_element.text.strip()
                print(parent_text)

                # Find all radio elements and their corresponding labels or child elements' text
                radio_elements = parent_element.find_elements(By.XPATH, './/input[@type="radio"]')
                for radio in radio_elements:
                    try:
                        label_text = radio.find_element(By.XPATH, './following-sibling::label').text
                        print(f"[{radio.get_attribute('value')}] {label_text}")
                    except NoSuchElementException:
                        child_text = radio.text
                        print(f"[{radio.get_attribute('value')}] {child_text}")

                # Prompt the user to select a radio option
                response = input("Enter the option: ")

                # Find the corresponding radio element based on the response
                radio_element = driver.find_element(By.XPATH, f'//label[contains(text(), "{response}")]/input[@type="radio"]')
                radio_element.click()


                print(f"\nLabel/Text: {label_text}\nInput Type: {input_type}\nValue given: {response}\n")

                import pdb;pdb.set_trace()

    select_options()


    import pdb;pdb.set_trace()



# Call the function to initialize the information
get_web()

